### Задание 2
Вам необходимо построить модель, которая на основании данных, поступающих каждую минуту, определяют качество продукции, производимое на обжиговой машине.
Обжиговая машина представляет собой агрегат, состоящий из 5 одинаковых по размеру камер, в каждой камере установлено по 3 датчика температур. Кроме этого, для данной задачи Вы собрали данные о высоте слоя сырья и его влажности. Высота слоя и влажность измеряются при входе сырья в машину. Сырье проходит через обжиговую машину за час.

Качество продукции измеряется в лаборатории по пробам, которые забираются каждый час, данные по известным анализам содержатся в файле Y_train.csv. В файле указано время забора пробы, проба забирается на выходе из обжиговой машины.
Вы договорились с заказчиком, что оценкой модели будет являться показатель MAE, для оценки модели необходимо сгенерировать предсказания за период, указанный в файле Y_submit.csv (5808 предиктов).

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import lightgbm as lgb
from catboost import CatBoostRegressor
import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
import eli5
import os
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [2]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [3]:
PATH = ''
SEED = 42

Загрузка данных

In [4]:
df = pd.read_csv(os.path.join(PATH, 'X_data.csv'), sep=';', index_col=[0], parse_dates=[0])
df.sort_index(inplace=True)

In [7]:
y = pd.read_csv(os.path.join(PATH, 'Y_train.csv'), sep=';', header=None, names=['y'], index_col=[0], parse_dates=[0])
y.sort_index(inplace=True)

In [57]:
y_submit = pd.read_csv(os.path.join(PATH, 'Y_submit.csv'), sep=';', header=None, names=['y'], index_col=[0], parse_dates=[0])
y_submit.sort_index(inplace=True)

Изучение данных

In [12]:
df.head()

,T_data_1_1,T_data_1_2,T_data_1_3,T_data_2_1,T_data_2_2,T_data_2_3,T_data_3_1,T_data_3_2,T_data_3_3,T_data_4_1,T_data_4_2,T_data_4_3,T_data_5_1,T_data_5_2,T_data_5_3,H_data,AH_data
2015-01-01 00:00:00,212,210,211,347,353,347,474,473,481,346,348,355,241,241,243,167.85000,9.22000
2015-01-01 00:01:00,212,211,211,346,352,346,475,473,481,349,348,355,241,241,243,162.51000,9.22000
2015-01-01 00:02:00,212,211,211,345,352,346,476,473,481,352,349,355,242,241,242,164.99000,9.22000
2015-01-01 00:03:00,213,211,211,344,351,346,477,473,481,355,349,355,242,241,242,167.34000,9.22000
2015-01-01 00:04:00,213,211,211,343,350,346,478,473,482,358,349,355,243,241,242,163.04000,9.22000


In [13]:
df.sample(random_state=SEED)

,T_data_1_1,T_data_1_2,T_data_1_3,T_data_2_1,T_data_2_2,T_data_2_3,T_data_3_1,T_data_3_2,T_data_3_3,T_data_4_1,T_data_4_2,T_data_4_3,T_data_5_1,T_data_5_2,T_data_5_3,H_data,AH_data
2018-04-27 04:48:00,238,250,237,342,337,340,490,477,493,400,403,400,268,249,258,164.52000,7.93000


In [224]:
df.tail()

,T_data_1_1,T_data_1_2,T_data_1_3,T_data_2_1,T_data_2_2,T_data_2_3,T_data_3_1,T_data_3_2,T_data_3_3,T_data_4_1,T_data_4_2,T_data_4_3,T_data_5_1,T_data_5_2,T_data_5_3,H_data,AH_data
2018-12-31 23:56:00,271,261,265,353,359,353,481,449,491,325,328,328,277,276,280,157.20,8.44
2018-12-31 23:57:00,271,261,265,353,359,353,481,449,491,325,328,328,277,276,280,160.40,8.44
2018-12-31 23:58:00,271,261,265,353,359,353,481,449,491,325,328,328,277,276,280,160.14,8.44
2018-12-31 23:59:00,271,261,265,353,359,353,481,449,491,325,328,328,277,276,280,162.96,8.44
2019-01-01 00:00:00,271,261,265,353,359,353,481,449,491,325,328,328,277,276,280,159.73,7.35


In [225]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2103841 entries, 2015-01-01 00:00:00 to 2019-01-01 00:00:00
Data columns (total 17 columns):
T_data_1_1    int64
T_data_1_2    int64
T_data_1_3    int64
T_data_2_1    int64
T_data_2_2    int64
T_data_2_3    int64
T_data_3_1    int64
T_data_3_2    int64
T_data_3_3    int64
T_data_4_1    int64
T_data_4_2    int64
T_data_4_3    int64
T_data_5_1    int64
T_data_5_2    int64
T_data_5_3    int64
H_data        float64
AH_data       float64
dtypes: float64(2), int64(15)
memory usage: 288.9 MB


In [228]:
df.isnull().sum()

T_data_1_1    0
T_data_1_2    0
T_data_1_3    0
T_data_2_1    0
T_data_2_2    0
T_data_2_3    0
T_data_3_1    0
T_data_3_2    0
T_data_3_3    0
T_data_4_1    0
T_data_4_2    0
T_data_4_3    0
T_data_5_1    0
T_data_5_2    0
T_data_5_3    0
H_data        0
AH_data       0
dtype: int64

Пропусков нет

In [425]:
y.head()

,y
2015-01-04 00:05:00,392
2015-01-04 01:05:00,384
2015-01-04 02:05:00,393
2015-01-04 03:05:00,399
2015-01-04 04:05:00,400


In [426]:
y.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 29184 entries, 2015-01-04 00:05:00 to 2018-05-03 23:05:00
Data columns (total 1 columns):
y    29184 non-null object
dtypes: object(1)
memory usage: 456.0+ KB


Целевой признак в `y` в интервале `2015-01-04 00:05:00 - 2018-05-03 23:05:00`. Измерения для пятой минуты каждого часа

Изменяю тип целевого признака на `int`

In [14]:
y['y'] = y['y'].astype('int')

In [456]:
y.describe()

,y
count,29184.00000
mean,402.80075
std,46.27323
min,221.00000
25%,372.00000
50%,408.00000
75%,439.00000
max,505.00000


В целевом признаке нет отрицательных и других явно ошибочных значений.

In [432]:
y_submit.head()

,y
2018-05-04 00:05:00,420
2018-05-04 01:05:00,420
2018-05-04 02:05:00,420
2018-05-04 03:05:00,420
2018-05-04 04:05:00,420


In [433]:
y_submit.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5808 entries, 2018-05-04 00:05:00 to 2018-12-31 23:05:00
Data columns (total 1 columns):
y    5808 non-null object
dtypes: object(1)
memory usage: 90.8+ KB


В `y_submit` временной интервал `2018-05-04 00:05:00 - 2018-12-31 23:05:00`. Измерения для пятой минуты каждого часа

Оставляю в датасете строки в пределах времени, для которого есть значения целевого признака в `y` и `y_submit`, с учётом отступа на один час

In [15]:
df = df[(df.index > '2015-01-03 23:05:00') & ('2018-12-31 23:05:00' > df.index)]

In [435]:
df.describe()

,T_data_1_1,T_data_1_2,T_data_1_3,T_data_2_1,T_data_2_2,T_data_2_3,T_data_3_1,T_data_3_2,T_data_3_3,T_data_4_1,T_data_4_2,T_data_4_3,T_data_5_1,T_data_5_2,T_data_5_3,H_data,AH_data
count,2099519.00000,2099519.00000,2099519.00000,2099519.00000,2099519.00000,2099519.00000,2099519.00000,2099519.00000,2099519.00000,2099519.00000,2099519.00000,2099519.00000,2099519.00000,2099519.00000,2099519.00000,2099519.00000,2099519.00000
mean,250.15040,250.05470,250.21752,349.79131,349.74010,349.84804,501.13023,501.07512,501.19513,349.31368,349.46074,350.03675,249.75100,249.61346,249.69282,174.74420,7.49910
std,32.12517,30.82454,30.69141,42.28482,40.65657,38.00820,63.29571,63.40511,62.26561,39.18773,39.17930,40.21692,30.74805,30.50129,30.76276,14.42954,1.14788
min,-198.00000,-122.00000,-107.00000,-703.00000,-958.00000,-191.00000,-775.00000,-759.00000,-613.00000,-514.00000,-471.00000,-609.00000,-89.00000,-125.00000,-163.00000,141.49000,2.89000
25%,229.00000,229.00000,229.00000,328.00000,328.00000,328.00000,463.00000,464.00000,464.00000,327.00000,328.00000,328.00000,229.00000,229.00000,229.00000,162.48000,6.73000
50%,250.00000,250.00000,250.00000,350.00000,350.00000,350.00000,502.00000,502.00000,502.00000,349.00000,350.00000,350.00000,249.00000,250.00000,250.00000,174.44000,7.51000
75%,272.00000,272.00000,272.00000,372.00000,372.00000,372.00000,538.00000,537.00000,538.00000,372.00000,372.00000,372.00000,270.00000,271.00000,271.00000,187.03000,8.27000
max,724.00000,762.00000,665.00000,1302.00000,1179.00000,889.00000,1587.00000,2505.00000,1319.00000,1177.00000,1244.00000,944.00000,905.00000,738.00000,624.00000,207.83000,11.84000


Отрицательные значения в данных об измерении температуры.

Заменю значения меньше 100 градусов на среднее по двум другим датчикам в той же камере. 

In [16]:
mean_list = []
for i in range(1, 6):
    df[f'{i}_mean_2_3'] = (df[f'T_data_{i}_2'] + df[f'T_data_{i}_3']) / 2
    mean_list.append(f'{i}_mean_2_3')
    df[f'{i}_mean_1_3'] = (df[f'T_data_{i}_1'] + df[f'T_data_{i}_3']) / 2
    mean_list.append(f'{i}_mean_1_3')
    df[f'{i}_mean_1_2'] = (df[f'T_data_{i}_1'] + df[f'T_data_{i}_2']) / 2
    mean_list.append(f'{i}_mean_1_2')

In [17]:
for i in range(1, 6):
    df.loc[df[f'T_data_{i}_1'] < 100, f'T_data_{i}_1'] = df.loc[df[f'T_data_{i}_1'] < 100, f'{i}_mean_2_3']
    df.loc[df[f'T_data_{i}_2'] < 100, f'T_data_{i}_2'] = df.loc[df[f'T_data_{i}_2'] < 100, f'{i}_mean_1_3']
    df.loc[df[f'T_data_{i}_3'] < 100, f'T_data_{i}_3'] = df.loc[df[f'T_data_{i}_3'] < 100, f'{i}_mean_1_2']

In [18]:
df = df.drop(mean_list, axis=1)

Так как целевой признак измеряется для пятой минуты каждого часа, а в основном датасете содержатся данные за каждую минуту, то необходимо изменить временной интервал. Для этого я смещу индексы в основном датасете на пять минут назад. Затем проведу ресемплирование по часу с усреднением значений и верну назад настоящие индексы. Таким образом интервал 60 минут с `2015-01-04 22:05:00` по `2015-01-04 23:04:00` будет соответствовать индексу времени `2015-01-04 23:05:00`

In [19]:
delta5_idx_minus = pd.date_range('2015-01-03 23:01:00', '2018-12-31 22:59:00', freq='1T')

In [20]:
df.reset_index(inplace=True)

In [21]:
df.set_index(delta5_idx_minus, inplace=True)

In [22]:
df_h = df.resample('1H').mean()

In [23]:
delta5_idx_plus = pd.date_range('2015-01-04 00:05:00', '2018-12-31 23:05:00', freq='1H')

In [24]:
df_h.reset_index(inplace=True)
df_h.set_index(delta5_idx_plus, inplace=True)

In [25]:
df_h = df_h.drop('index', axis=1)

Создаю датасеты `df_train` и `df_submit` в соответствии с временными интервалами в `y_train` и `y_submit`

In [26]:
df_train = df_h[(df_h.index >= '2015-01-04 00:05:00') & ('2018-05-03 23:05:00' >= df_h.index)]

In [27]:
(df_train.shape), (y.shape)

((29184, 17), (29184, 1))

In [28]:
df_submit = df_h[(df_h.index >= '2018-05-04 00:05:00') & ('2018-12-31 23:05:00' >= df_h.index)]

In [29]:
(df_submit.shape), (y_submit.shape)

((5808, 17), (5808, 1))

Разделение датасета `df_train` на обучающую и тестовую выборки в соотношении 70:30

In [30]:
X_train, X_test, y_train, y_test = train_test_split(df_train, y, shuffle=False, test_size=0.3)

### Обучение и тестирование моделей

Поиск гиперпарметров будет осуществляться с помощью `GridSearchCV` с кросс-валидацией на 5 подвыборках

In [31]:
kf = KFold(n_splits=5, random_state=SEED, shuffle=False)

#### Линейная регрессия

In [33]:
parameters = {'fit_intercept':[True,False], 'normalize':[True,False]}
grid_lr = GridSearchCV(LinearRegression(), parameters, cv=kf, scoring='neg_mean_absolute_error')
grid_lr.fit(X_train, y_train)
params = grid_lr.best_params_
print('MAE на кросс-валидации:', abs(grid_lr.best_score_))
print('Гиперпараметры лучшей модели:', params)
lr = LinearRegression(**params)
lr.fit(X_train, y_train)

MAE на кросс-валидации: 13.416187394284895
Гиперпараметры лучшей модели: {'fit_intercept': True, 'normalize': True}


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=True)

#### Случайный лес

In [34]:
tree_params = {'n_estimators': [1000], 
               'max_depth': [4, 6],
               'min_samples_split': [2, 5, 8, 10]}

In [35]:
grid_rfr = GridSearchCV(RandomForestRegressor(random_state=SEED), 
                        tree_params, n_jobs=-1, refit=False, cv=kf, scoring='neg_mean_absolute_error') 
grid_rfr.fit(X_train, y_train)
print('MAE на кросс-валидации: ', abs(grid_rfr.best_score_))
params = grid_rfr.best_params_
print('Гиперпараметры лучшей модели:', params)
rfr = RandomForestRegressor(**params, random_state=SEED)
rfr.fit(X_train, y_train)

MAE на кросс-валидации:  12.197622860281523
Гиперпараметры лучшей модели: {'max_depth': 6, 'min_samples_split': 2, 'n_estimators': 1000}


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=6,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=None,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

#### Градиентный бустинг

#### XGBoost

In [36]:
param_grid_xgb = {'learning_rate': [0.03, 0.05, 0.1, 0.2],
                  'max_depth': [4, 6],
                  'n_estimators': [1000]}

In [37]:
grid_xgbr = GridSearchCV(xgb.XGBRegressor(random_state=SEED, verbosity=0), 
                    param_grid_xgb, n_jobs=-1, cv=kf, scoring='neg_mean_absolute_error', refit=False) 
grid_xgbr.fit(X_train, y_train)
params = grid_xgbr.best_params_
print('MAE на кросс-валидации:', abs(grid_xgbr.best_score_))
print('Гиперпараметры лучшей модели:', params)
xgbr= xgb.XGBRegressor(**params, random_state=SEED)
xgbr.fit(X_train, y_train)

MAE на кросс-валидации: 8.34908309318045
Гиперпараметры лучшей модели: {'learning_rate': 0.05, 'max_depth': 6, 'n_estimators': 1000}
[11:40:36] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0,
       importance_type='gain', learning_rate=0.05, max_delta_step=0,
       max_depth=6, min_child_weight=1, missing=None, n_estimators=1000,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=42,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=1, verbosity=1)

#### LightGBM

In [38]:
param_grid = {'n_estimators': [1000], 
              'learning_rate': [0.05, 0.1],
              'max_depth': [4, 6],
              'num_leaves': [20, 30, 50]}

In [39]:
grid_lgbm = GridSearchCV(lgb.LGBMRegressor(random_state=SEED), 
                    param_grid, n_jobs=-1, cv=kf, scoring='neg_mean_absolute_error', refit=False) 
grid_lgbm.fit(X_train, y_train)
params = grid_lgbm.best_params_
print('MAE на кросс-валидации:', abs(grid_lgbm.best_score_))
print('Гиперпараметры лучшей модели:', params)
lgbm = lgb.LGBMRegressor(**params, random_state=SEED)
lgbm.fit(X_train, y_train)

MAE на кросс-валидации: 8.132341792850776
Гиперпараметры лучшей модели: {'learning_rate': 0.05, 'max_depth': 6, 'n_estimators': 1000, 'num_leaves': 30}


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       importance_type='split', learning_rate=0.05, max_depth=6,
       min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
       n_estimators=1000, n_jobs=-1, num_leaves=30, objective=None,
       random_state=42, reg_alpha=0.0, reg_lambda=0.0, silent=True,
       subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

#### CatBoost

In [40]:
param_grid_ctb = {'n_estimators': [1000],  
                  'learning_rate': [0.03, 0.05, 0.07, 0.1],
                  'depth': [4, 6]}

In [41]:
grid_ctb = GridSearchCV(CatBoostRegressor(random_state=SEED, silent=True), 
                    param_grid_ctb, n_jobs=-1, cv=kf, scoring='neg_mean_absolute_error', refit=False) 
grid_ctb.fit(X_train, y_train)
print('MAE на кросс-валидации: ', abs(grid_ctb.best_score_))
params = grid_ctb.best_params_
print('Гиперпараметры лучшей модели:', params)
ctb = CatBoostRegressor(**params, random_state=SEED, silent=True)
ctb.fit(X_train, y_train)

MAE на кросс-валидации:  7.53925700447485
Гиперпараметры лучшей модели: {'depth': 6, 'learning_rate': 0.07, 'n_estimators': 1000}


#### Значение MAE на кросс-валидации:
- CatBoostRegressor - 7.5, 
- LGBMRegressor - 8.1, 
- XGBRegressor - 8.3, 
- RandomForestRegressor - 12.2, 
- LinearRegression - 13.4

#### Тестирование моделей

In [42]:
def test_model(model):
    pred_test = model.predict(X_test)
    print('MAE на тестовой выборке:', mean_absolute_error(y_test, pred_test))

Линейная регрессия

In [43]:
test_model(lr)

MAE на тестовой выборке: 13.319455502837089


Случайный лес

In [44]:
test_model(rfr)

MAE на тестовой выборке: 12.11010439498051


XGBoost

In [45]:
test_model(xgbr)

MAE на тестовой выборке: 8.30142179500463


LightGBM

In [46]:
test_model(lgbm)

MAE на тестовой выборке: 8.207631915748866


CatBoost

In [47]:
test_model(ctb)

MAE на тестовой выборке: 7.505597019081364


Значимость признаков для лучшей модели CatBoost

In [48]:
eli5.explain_weights_catboost(ctb, top=17)

Weight,Feature
0.2183,T_data_3_1
0.2175,T_data_3_3
0.2098,T_data_3_2
0.0855,H_data
0.0553,T_data_5_1
0.0391,T_data_5_2
0.0340,T_data_1_2
0.0321,T_data_5_3
0.0310,T_data_1_3
0.0199,T_data_1_1


### Отчёт

#### 1) Датасет

На этапе предобработки данных значения температуры меньше 100 градусов были заменены на среднее по двум другим датчикам в той же камере. Далее данные ресемплированы по часу с усреднением.

- Итоговый датасет состоит из 18 признаков и 29184 строк.
- Временной интервал `2015-01-04 00:05:00 - 2018-05-03 23:05:00`
- Обучающая и тестовая выборка в соотношении 70:30 - 20428 и 8756 строк.
- 15 признаков - усредненные показания температуры трёх датчиков в каждой из пяти камер
- 1 признак - усредненная высота слоя
- 1 признак - усредненная влажность сырья
- 1 целевой признак - качество продукции

#### 2) Модели
- Обучены пять моделей. Результаты MAE на тестовой выборке: CatBoostRegressor - 7.5, LGBMRegressor - 8.2, XGBRegressor - 8.3, RandomForestRegressor - 12.1, LinearRegression - 13.3
- На тестовой выборке лучшего значения MAE 7.5 удалось достичь используя модель CatBoostRegressor с гиперпараметрами {'depth': 6, 'learning_rate': 0.07, 'n_estimators': 1000} 
- Наиболее значимые признаки для лучшей модели: показания трёх датчиков температуры в камере 3 - суммарно около 65%, высота слоя - 8,5%

### Предсказание

Использую модель CatBoostRegressor с гиперпараметрами {'depth': 6, 'learning_rate': 0.07, 'n_estimators': 1000}. Обучение на всём датасете.

In [58]:
ctb_submit = CatBoostRegressor(random_state=SEED, silent=True, depth=6, learning_rate=0.07, n_estimators=1000)
ctb_submit.fit(df_train, y)

In [59]:
y_submit['predictions'] = ctb_submit.predict(df_submit)

In [60]:
y_submit = y_submit.drop('y', axis=1)

In [61]:
y_submit.head()

,predictions
2018-05-04 00:05:00,449.55324
2018-05-04 01:05:00,442.89885
2018-05-04 02:05:00,436.38604
2018-05-04 03:05:00,412.81239
2018-05-04 04:05:00,411.77652


In [64]:
y_submit.to_csv('submit.csv')